In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-21 18:42:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2021-12-21 18:42:27 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

### Deliverable 2

In [31]:
#filter the dataset for total_votes being greater than or equal to 20
vote_count = df.filter("total_votes >= 20")
vote_count.count()

224930

In [6]:
#New DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
filter_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
filter_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44783596|R31EUR60FV3BU5|B00ZGPZR9I|     514136181|  Wodehouse in Exile|       Video DVD|          4|           32|         39|   N|                N|Unfair Accusation...|Amazed after Wode...| 2015-08-31|
|         US|   12720421|R3MDX24QDAT0OW|B00ZCHTQGA|     779817400|Northmen - A Viki...|       Video DVD|          4|    

In [10]:
#New DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
filter_table.filter(filter_table["vine"] == "Y").show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27249597|R3PBQ665MBL8CP|B00E1LT41K|     592331046|The White Queen: ...|       Video DVD|          3|           66|         72|   Y|                N|An enjoyable watc...|I enjoyed this TV...| 2014-03-27|
|         US|   37876368|R3EMD30K7MNOZ0|B00A27OMKU|     678954891|Shiva Rea: Yoga i...|       Video DVD|          2|    

In [11]:
#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
filter_table.filter(filter_table["vine"] == "N").show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44783596|R31EUR60FV3BU5|B00ZGPZR9I|     514136181|  Wodehouse in Exile|       Video DVD|          4|           32|         39|   N|                N|Unfair Accusation...|Amazed after Wode...| 2015-08-31|
|         US|   12720421|R3MDX24QDAT0OW|B00ZCHTQGA|     779817400|Northmen - A Viki...|       Video DVD|          4|    

In [12]:
# Number of five star reviews
five_star_rating = filter_table.filter(filter_table["star_rating"]== 5)
five_star_rating.cache()

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [13]:
# Percentage of five star reviews
five_star_rating.count() / filter_table.count()

0.5154870616511169

In [32]:
#percentage of paid reviews
five_star_rating.filter(five_star_rating["vine"] == "Y").count()/filter_table.filter(filter_table["vine"]== "Y").count()

0.1836734693877551

In [33]:
#percentage of unpaid reviews
five_star_rating.filter(five_star_rating["vine"] == "N").count()/filter_table.filter(filter_table["vine"]== "N").count()

0.5155944517833554

In [18]:
# Total count of five star reviews
five_star_rating.count()

78070

In [19]:
# Total amount of reviews
filter_table.count()

151449

In [26]:
# Total amount of filtered data non vine viewers
filter_table.filter(filter_table["vine"]== "N").count()

151400

In [27]:
# Total amount of filtered data vine viewers
filter_table.filter(filter_table["vine"]== "Y").count()

49

In [28]:
# Total amount of 5 stars vine reviews
five_star_rating.filter(five_star_rating["vine"] == "Y").count()

9

In [30]:
# Total amount of 5 stars non-vine reviews
five_star_rating.filter(five_star_rating["vine"] == "N").count()

78061